Install PyTorch, per the instruction: https://pytorch.org/

In [27]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import os
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from torchvision import transforms
import cv2


Check if PyTorch uses GPU; if not, try: \
&emsp; 1. PyTorch only uses NVDIA, check if NVDIA in Task Manager -> Performance -> GPU \
&emsp; 2. If YES to 1, try command "nvidia-smi" to see driver version \
&emsp; 3. If too low, download and update driver from NVDIA website

In [2]:
torch.cuda.is_available()

False

Two commonly-used functions: \
&emsp; 1. dir(torch) \
&emsp; 2. help(torch)

In [3]:
dir(torch.cuda.is_available)
help(torch.cuda.is_available)

Help on function is_available in module torch.cuda:

is_available() -> bool
    Returns a bool indicating if CUDA is currently available.



PyTorch, to load datasets: \
&emsp; 1. Dataset: Provide a way to collect data and its label; \
&emsp; 2. Dataloader: Provide different data format for networks; \
&emsp; 3. Sample Datasets Download: https://download.pytorch.org/tutorial/hymenoptera_data.zip

In [4]:
"""
Create class MyData from Dataset, overwritting __init__ & __getitem__
"""
class MyData(Dataset):
    
    def __init__(self, root_dir, train_val_dir, label_dir):
        self.root_dir = root_dir
        self.train_val_dir = train_val_dir
        self.label_dir = label_dir
        self.path = os.path.join(self.root_dir, self.train_val_dir, self.label_dir)
        self.img_path = os.listdir(self.path)

    def __getitem__(self, index):
        img_name = self.img_path[index]
        img_item_path = os.path.join(self.root_dir, self.train_val_dir, self.label_dir, img_name)
        img = Image.open(img_item_path)
        label = self.label_dir
        return img, label
    
    def __len__(self):
        return len(self.img_path)

In [5]:
"""
Train - Ants
"""
root_dir = "C:\\Hunting\\2023\\data_personal\\hymenoptera"
train_val_dir = "train"
label_dir = "ants"
ants = MyData(root_dir=root_dir,train_val_dir=train_val_dir, label_dir=label_dir)
img, label = ants[0]
img.show()

In [6]:
"""
Train - Bees
"""
root_dir = "C:\\Hunting\\2023\\data_personal\\hymenoptera"
train_val_dir = "train"
label_dir = "bees"
bees = MyData(root_dir=root_dir,train_val_dir=train_val_dir, label_dir=label_dir)
img, label = bees[0]
img.show()

In [7]:
train_dataset = ants + bees
train_dataset[123][0].show()

Tensorboard - Display Images \
&emsp; 1. tensorboard --logdir=logs --port=6007 \
&emsp; 2. used for training loss display - One image per step

In [8]:
writer = SummaryWriter("logs")
# writer.add_image()
for i in range(100):
    writer.add_scalar("y=x", i, i)
writer.close()

In [9]:
img, label = ants[0]
img_array = np.array(img)
print(f"Type of Image: {type(img_array)}")
print(f"Shape of Image: {img_array.shape}")

Type of Image: <class 'numpy.ndarray'>
Shape of Image: (512, 768, 3)


In [10]:
writer = SummaryWriter("logs")
writer.add_image("test", img_array, 1, dataformats='HWC')
for i in range(100):
    writer.add_scalar("y=2x", 2*i, i)
writer.close()

Transform - Image Reformat \
&emsp; 1. ToTensor(object): a ''PIL Image'' or ''numpy.ndarray'' to tensor \
&emsp; &emsp; 1.1 How to use tranforms \
&emsp; &emsp; 1.2 Why need Tensor data structure \
&emsp; 2. Normalize \
&emsp; 3. Resize \
&emsp; 4. Compose

In [16]:
"""
PIL Image
"""
tensor_trains = transforms.ToTensor()
tensor_img = tensor_trains(img)
print(type(tensor_img))
print(tensor_img.shape)

<class 'torch.Tensor'>
torch.Size([3, 512, 768])


In [19]:
writer.add_image("Tensor_img", tensor_img)
writer.close()

In [20]:
"""
Normalize
"""
trans_norms = transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
img_norms = trans_norms(tensor_img)
writer.add_image("Tensor_img_norm", img_norms)
writer.close()

In [21]:
"""
Resize
"""
trans_resize = transforms.Resize((512, 512))
img_resize = trans_resize(img)
img_resize = tensor_trains(img_resize)
writer.add_image("Tensor_img_resize", img_resize)
writer.close()

In [22]:
"""
Compose - resie - 2
"""
trans_resize_2 = transforms.Resize(512)
trans_compose = transforms.Compose([trans_resize_2, tensor_trains])
img_resize_2 = trans_compose(img)
# img_resize = tensor_trains(img_resize)
writer.add_image("Tensor_img_resize_2", img_resize_2)
writer.close()

Torchvision of the whole dataset

In [23]:
from torchvision import datasets

In [24]:
train_set = datasets.CIFAR10(root=root_dir, train=True, download=True)
test_set = datasets.CIFAR10(root=root_dir, train=False, download=True)

100.0%


Extracting C:\Hunting\2023\data_personal\hymenoptera\cifar-10-python.tar.gz to C:\Hunting\2023\data_personal\hymenoptera
Files already downloaded and verified


In [26]:
print(test_set.classes)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [29]:
dataset_transform = transforms.Compose([
    transforms.ToTensor()
])

train_set = datasets.CIFAR10(root=root_dir, train=True, transform=dataset_transform, download=True)
test_set = datasets.CIFAR10(root=root_dir, train=False, transform=dataset_transform, download=True)

100.0%


Extracting C:\Hunting\2023\data_personal\hymenoptera\cifar-10-python.tar.gz to C:\Hunting\2023\data_personal\hymenoptera
Files already downloaded and verified


DataLoader

In [30]:
test_loader = DataLoader(dataset=test_set, batch_size=4, shuffle=True, num_workers=0, drop_last=False)

In [32]:
for epoch in range(2):
    step = 0
    for data in test_loader:
        imgs, targets = data
        # print(imgs.shape)
        # print(targets)
        writer.add_images("test_data", imgs, step)
        step += 1
    writer.close()